# I - Fluxonium Qubit using SCQubits

In [2]:
import scqubits as sc

In [3]:
fluxonium = sc.Fluxonium(EJ = 8.9,
                               EC = 2.5,
                               EL = 0.5,
                               flux = 0,
                               cutoff = 110)


In [4]:
evals_scqubits = fluxonium.eigenvals()
evals_scqubits -= evals_scqubits[0]
evals_scqubits

array([ 0.        ,  8.75844526,  8.97955262, 11.00203435, 16.93299002,
       19.57185963])

# II - Fluxonium Qubit using QuTiP

In [5]:
import qutip as qt
import numpy as np
from scipy.constants import hbar, e

In [6]:
truncation_level = 10 # max dimension to keep in the matrices

In [7]:
creation = qt.create(truncation_level)  # Creation and destruction operators
destruction = qt.destroy(truncation_level)

We have the following Hamiltonian for the circuit (that can be found by wrinting the lagrangian, etc.) :

$$
H = \frac 1 {2C} q^2 + \frac 1 {2L} \phi^2 - \frac {\phi_0^2} {L_C} \cos \bigg(\frac \phi {\phi_0} \bigg)
$$

Where $$q = C\dot\phi$$ $$\phi_0 = \frac \hbar {2e}$$

If we now replace $C$ and $L$ with $C = \frac {e^2} {2E_C}$ and $L = \frac {\phi_0^2} {E_L}$, and use $E_J = \frac {\phi_0^2} {L_C}$, we get the following expression :

$$
H = \frac {E_C} {e^2} q^2 + \frac {E_L} {2\phi_0^2} \phi^2 - E_J \cos \bigg(\frac \phi {\phi_0} \bigg)
$$

Let us define :
$$\hat n := \frac {\hat q} {2e}$$
$$\hat \varphi := \frac {\hat \phi} {\phi_0}$$

So that : 
$$
\hat H = 4E_C \hat n^2 + \frac {E_L} 2 \hat\varphi^2 - E_J \cos (\hat \varphi )
$$

Finally, let us define the destruction operator :
$$ \hat a := \frac 1 2 \bigg(\frac {E_L} {2E_C}\bigg)^{\frac 1 4}\hat\varphi + i\bigg(\frac {2E_C} {E_L} \bigg)^{\frac 1 4}\hat n $$

If we define $\varepsilon := \big(\frac {E_L} {2E_C}\big)^{\frac 1 4}$, this boils down to :
$$ \hat a = \frac 1 2 \varepsilon\hat\varphi + i\frac 1 {\varepsilon}\hat n $$

We then have, since $[\hat \varphi, \hat n] = i$ $$[\hat a, \hat a^\dag] = 1$$

And the Hamiltonian can be rewritten as :
$$
\hat H = \hbar \omega \bigg(\hat a^\dag \hat a + \frac 1 2\bigg) - E_J \cos \bigg( \frac 1 \varepsilon (\hat a + \hat a^\dag) \bigg)
$$

as $\hbar\omega = \sqrt{8E_CE_L}$ if $\omega = \frac 1 {\sqrt{LC}}$.

This is the formula that we will implement. If we wish, we can also add an external flux $\varphi_{ext}$ between $0$ and $2\pi$ inside the cosinus.

In [8]:
def hamiltonian_qutip(EL,EC,EJ) :
    '''
    A simple funciton that returns the hamiltonian of a fluxonium circuit
    '''

    epsilon = (EL/(2*EC))**(1/4)

    return np.sqrt(8*EC*EL) * (creation*destruction + 1/2) - EJ * ((creation + destruction)/epsilon).cosm()

In [9]:
evals_qutip = hamiltonian_qutip(0.5, 2.5, 8.9).eigenenergies()
evals_qutip -= evals_qutip[0]
evals_qutip

array([ 0.        ,  8.03391431,  8.26466763,  9.70413078, 16.32474141,
       17.61579809, 27.31666855, 30.71176171, 33.39536956, 35.04723056])

# III - Fluxonium Qubit using Numpy

In [10]:
import numpy as np
import numpy.linalg as alg
from scipy.linalg import cosm

In [11]:
truncation_level = 10 # max dimension to keep in the matrices

Here, we will use the fact that the eigenvalues of $\hat H$ can be explored using the jump operators, meaning that $\hat a |n\rangle$ is associated to the eigenvalue $n-1$, and $\hat a^\dag |n\rangle$ to the eigenvalue $n+1$

We have :
$$
\begin{aligned}
\|\hat a | n \rangle \|^2 & = \langle n | \hat a^\dag \hat a | n \rangle \\
& = \langle n |n| n \rangle \\
& = n
\end{aligned}
$$
Same goes for the other one.
Therefore, if we normalize $\hat a |n\rangle$ and $\hat a^\dag |n\rangle$, we get the relationships :

$$
\begin{equation}
    \begin{cases}
        \hat a |n\rangle = \sqrt n | n-1 \rangle \\
        \hat a^\dag |n\rangle= \sqrt {n+1} | n+1 \rangle
    \end{cases}\,.
\end{equation}
$$

From these relationships we obtain the form of the $\hat a$ and $\hat a^\dag$ matrices, truncated to order $d$, in the Fock basis.

In [12]:
destruction = np.zeros((truncation_level,truncation_level))

for n in range (1,truncation_level): destruction[n-1,n] = np.sqrt(n)

creation = destruction.T #Since it is a real matrix in the Fock basis.

In [13]:
def hamiltonian_numpy(EL,EC,EJ) :
    '''
    A simple funciton that returns the hamiltonian of a fluxonium circuit
    '''

    epsilon = (EL/(2*EC))**(1/4)

    return np.sqrt(8*EC*EL) * (np.dot(creation,destruction) + 1/2) - EJ * cosm((creation + destruction)/epsilon)

In [14]:
evals_numpy = np.sort(alg.eig(hamiltonian_numpy(0.5, 2.5, 8.9))[0])
evals_numpy -= evals_numpy[0]
evals_numpy

array([ 0.        ,  7.74482978,  9.29602638, 12.17114858, 16.13322538,
       17.29093152, 27.08296197, 30.73916618, 33.79810346, 43.9412028 ])